# Mạng tạo sinh

Mạng Nơ-ron Hồi quy (RNNs) và các biến thể tế bào có cổng của chúng như Tế bào Bộ nhớ Ngắn Dài (LSTMs) và Đơn vị Hồi quy Có cổng (GRUs) cung cấp một cơ chế để mô hình hóa ngôn ngữ, tức là chúng có thể học cách sắp xếp từ và đưa ra dự đoán cho từ tiếp theo trong một chuỗi. Điều này cho phép chúng ta sử dụng RNNs cho các **nhiệm vụ tạo sinh**, chẳng hạn như tạo văn bản thông thường, dịch máy, và thậm chí là chú thích hình ảnh.

Trong kiến trúc RNN mà chúng ta đã thảo luận ở đơn vị trước, mỗi đơn vị RNN tạo ra trạng thái ẩn tiếp theo như một đầu ra. Tuy nhiên, chúng ta cũng có thể thêm một đầu ra khác vào mỗi đơn vị hồi quy, điều này cho phép chúng ta xuất ra một **chuỗi** (có độ dài bằng với chuỗi ban đầu). Hơn nữa, chúng ta có thể sử dụng các đơn vị RNN không nhận đầu vào ở mỗi bước, mà chỉ nhận một vector trạng thái ban đầu, sau đó tạo ra một chuỗi các đầu ra.

Trong notebook này, chúng ta sẽ tập trung vào các mô hình tạo sinh đơn giản giúp chúng ta tạo văn bản. Để đơn giản, hãy xây dựng **mạng cấp độ ký tự**, mạng này tạo văn bản từng chữ cái một. Trong quá trình huấn luyện, chúng ta cần lấy một tập hợp văn bản và chia nó thành các chuỗi ký tự.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Xây dựng từ vựng ký tự

Để xây dựng mạng sinh ở cấp độ ký tự, chúng ta cần chia văn bản thành các ký tự riêng lẻ thay vì các từ. Lớp `TextVectorization` mà chúng ta đã sử dụng trước đây không thể làm điều này, vì vậy chúng ta có hai lựa chọn:

* Tự tải văn bản và thực hiện việc phân tách 'thủ công', như trong [ví dụ chính thức của Keras này](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Sử dụng lớp `Tokenizer` để phân tách ở cấp độ ký tự.

Chúng ta sẽ chọn phương án thứ hai. `Tokenizer` cũng có thể được sử dụng để phân tách thành từ, vì vậy bạn có thể dễ dàng chuyển đổi từ phân tách cấp ký tự sang cấp từ.

Để thực hiện phân tách ở cấp độ ký tự, chúng ta cần truyền tham số `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Chúng tôi cũng muốn sử dụng một token đặc biệt để biểu thị **kết thúc chuỗi**, mà chúng tôi sẽ gọi là `<eos>`. Hãy thêm nó thủ công vào từ vựng:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Huấn luyện RNN sinh để tạo tiêu đề

Cách chúng ta sẽ huấn luyện RNN để tạo tiêu đề tin tức như sau. Ở mỗi bước, chúng ta sẽ lấy một tiêu đề, tiêu đề này sẽ được đưa vào RNN, và với mỗi ký tự đầu vào, chúng ta sẽ yêu cầu mạng tạo ra ký tự đầu ra tiếp theo:

![Hình ảnh minh họa việc RNN tạo ra từ 'HELLO'.](../../../../../translated_images/vi/rnn-generate.56c54afb52f9781d.webp)

Đối với ký tự cuối cùng của chuỗi, chúng ta sẽ yêu cầu mạng tạo ra token `<eos>`.

Điểm khác biệt chính của RNN sinh mà chúng ta sử dụng ở đây là chúng ta sẽ lấy đầu ra từ mỗi bước của RNN, chứ không chỉ từ ô cuối cùng. Điều này có thể thực hiện được bằng cách chỉ định tham số `return_sequences` cho ô RNN.

Do đó, trong quá trình huấn luyện, đầu vào của mạng sẽ là một chuỗi các ký tự được mã hóa với độ dài nhất định, và đầu ra sẽ là một chuỗi có cùng độ dài, nhưng được dịch chuyển một phần tử và kết thúc bằng `<eos>`. Minibatch sẽ bao gồm một số chuỗi như vậy, và chúng ta sẽ cần sử dụng **padding** để căn chỉnh tất cả các chuỗi.

Hãy tạo các hàm để chuyển đổi tập dữ liệu cho chúng ta. Vì chúng ta muốn thêm padding vào các chuỗi ở cấp độ minibatch, trước tiên chúng ta sẽ nhóm tập dữ liệu bằng cách gọi `.batch()`, và sau đó `map` nó để thực hiện chuyển đổi. Vì vậy, hàm chuyển đổi sẽ nhận toàn bộ minibatch làm tham số:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Một vài điều quan trọng mà chúng ta thực hiện ở đây:
* Đầu tiên, chúng ta trích xuất văn bản thực tế từ tensor chuỗi
* `text_to_sequences` chuyển đổi danh sách các chuỗi thành danh sách các tensor số nguyên
* `pad_sequences` thêm phần đệm vào các tensor đó để đạt độ dài tối đa
* Cuối cùng, chúng ta mã hóa one-hot tất cả các ký tự, đồng thời thực hiện việc dịch chuyển và thêm `<eos>`. Chúng ta sẽ sớm thấy lý do tại sao cần mã hóa one-hot các ký tự

Tuy nhiên, hàm này mang tính chất **Pythonic**, nghĩa là nó không thể được tự động chuyển đổi thành đồ thị tính toán của Tensorflow. Chúng ta sẽ gặp lỗi nếu cố gắng sử dụng hàm này trực tiếp trong hàm `Dataset.map`. Chúng ta cần bao bọc lời gọi Pythonic này bằng cách sử dụng trình bao `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Việc phân biệt giữa các hàm chuyển đổi Pythonic và Tensorflow có thể hơi phức tạp, và bạn có thể thắc mắc tại sao chúng ta không chuyển đổi tập dữ liệu bằng các hàm Python tiêu chuẩn trước khi truyền nó vào `fit`. Mặc dù điều này hoàn toàn có thể thực hiện, việc sử dụng `Dataset.map` có một lợi thế lớn, vì pipeline chuyển đổi dữ liệu được thực thi bằng đồ thị tính toán của Tensorflow, tận dụng khả năng tính toán của GPU và giảm thiểu nhu cầu truyền dữ liệu giữa CPU/GPU.

Bây giờ chúng ta có thể xây dựng mạng generator và bắt đầu huấn luyện. Nó có thể dựa trên bất kỳ cell hồi quy nào mà chúng ta đã thảo luận trong bài học trước (đơn giản, LSTM hoặc GRU). Trong ví dụ này, chúng ta sẽ sử dụng LSTM.

Vì mạng nhận các ký tự làm đầu vào và kích thước từ vựng khá nhỏ, chúng ta không cần lớp embedding, đầu vào được mã hóa one-hot có thể trực tiếp đi vào cell LSTM. Lớp đầu ra sẽ là một bộ phân loại `Dense` chuyển đổi đầu ra của LSTM thành các số token được mã hóa one-hot.

Ngoài ra, vì chúng ta đang xử lý các chuỗi có độ dài thay đổi, chúng ta có thể sử dụng lớp `Masking` để tạo một mặt nạ bỏ qua phần được đệm của chuỗi. Điều này không hoàn toàn cần thiết, vì chúng ta không quá quan tâm đến mọi thứ vượt quá token `<eos>`, nhưng chúng ta sẽ sử dụng nó để có thêm kinh nghiệm với loại lớp này. `input_shape` sẽ là `(None, vocab_size)`, trong đó `None` biểu thị chuỗi có độ dài thay đổi, và hình dạng đầu ra cũng là `(None, vocab_size)`, như bạn có thể thấy từ `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Tạo đầu ra

Bây giờ chúng ta đã huấn luyện mô hình, chúng ta muốn sử dụng nó để tạo một số đầu ra. Trước tiên, chúng ta cần một cách để giải mã văn bản được biểu diễn bằng một chuỗi số token. Để làm điều này, chúng ta có thể sử dụng hàm `tokenizer.sequences_to_texts`; tuy nhiên, nó không hoạt động tốt với việc mã hóa token ở cấp độ ký tự. Vì vậy, chúng ta sẽ lấy một từ điển các token từ tokenizer (gọi là `word_index`), xây dựng một bản đồ ngược, và tự viết hàm giải mã của riêng mình:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Bây giờ, chúng ta sẽ bắt đầu với một chuỗi `start`, mã hóa nó thành một dãy `inp`, và sau đó ở mỗi bước, chúng ta sẽ gọi mạng để suy luận ký tự tiếp theo.

Đầu ra của mạng `out` là một vector gồm `vocab_size` phần tử, đại diện cho xác suất của mỗi token, và chúng ta có thể tìm số token có xác suất cao nhất bằng cách sử dụng `argmax`. Sau đó, chúng ta thêm ký tự này vào danh sách các token đã được tạo, và tiếp tục quá trình tạo. Quá trình tạo một ký tự này được lặp lại `size` lần để tạo ra số lượng ký tự yêu cầu, và chúng ta sẽ kết thúc sớm khi gặp `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Lấy mẫu đầu ra trong quá trình huấn luyện

Vì chúng ta không có bất kỳ chỉ số hữu ích nào như *độ chính xác*, cách duy nhất để thấy rằng mô hình của chúng ta đang cải thiện là bằng cách **lấy mẫu** chuỗi được tạo ra trong quá trình huấn luyện. Để làm điều này, chúng ta sẽ sử dụng **callbacks**, tức là các hàm mà chúng ta có thể truyền vào hàm `fit`, và chúng sẽ được gọi định kỳ trong quá trình huấn luyện.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Ví dụ này đã tạo ra một văn bản khá tốt, nhưng vẫn có thể cải thiện thêm theo nhiều cách:

* **Thêm nội dung**. Chúng ta chỉ sử dụng tiêu đề cho nhiệm vụ này, nhưng bạn có thể thử nghiệm với toàn bộ văn bản. Hãy nhớ rằng RNN không xử lý tốt các chuỗi dài, vì vậy sẽ hợp lý hơn nếu chia chúng thành các câu ngắn hơn hoặc luôn huấn luyện trên một độ dài chuỗi cố định với giá trị được định trước `num_chars` (ví dụ, 256). Bạn có thể thử thay đổi ví dụ trên thành kiến trúc như vậy, sử dụng [hướng dẫn chính thức của Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) làm nguồn cảm hứng.

* **LSTM nhiều lớp**. Sẽ hợp lý nếu thử nghiệm với 2 hoặc 3 lớp tế bào LSTM. Như đã đề cập trong bài học trước, mỗi lớp LSTM sẽ trích xuất các mẫu nhất định từ văn bản, và trong trường hợp trình tạo cấp độ ký tự, chúng ta có thể kỳ vọng lớp LSTM thấp hơn chịu trách nhiệm trích xuất âm tiết, và các lớp cao hơn - từ và tổ hợp từ. Điều này có thể được thực hiện đơn giản bằng cách truyền tham số số lượng lớp vào hàm khởi tạo LSTM.

* Bạn cũng có thể muốn thử nghiệm với **đơn vị GRU** để xem loại nào hoạt động tốt hơn, và với **kích thước lớp ẩn khác nhau**. Lớp ẩn quá lớn có thể dẫn đến hiện tượng overfitting (ví dụ: mạng sẽ học chính xác văn bản), và kích thước nhỏ hơn có thể không tạo ra kết quả tốt.


## Tạo văn bản mềm và nhiệt độ

Trong định nghĩa trước của `generate`, chúng ta luôn chọn ký tự có xác suất cao nhất làm ký tự tiếp theo trong văn bản được tạo. Điều này dẫn đến việc văn bản thường "lặp lại" giữa các chuỗi ký tự giống nhau nhiều lần, như trong ví dụ sau:
```
today of the second the company and a second the company ...
```

Tuy nhiên, nếu chúng ta xem xét phân bố xác suất cho ký tự tiếp theo, có thể thấy rằng sự khác biệt giữa một vài xác suất cao nhất không lớn, ví dụ: một ký tự có xác suất là 0.2, ký tự khác là 0.19, v.v. Chẳng hạn, khi tìm ký tự tiếp theo trong chuỗi '*play*', ký tự tiếp theo có thể là khoảng trắng hoặc **e** (như trong từ *player*).

Điều này dẫn đến kết luận rằng không phải lúc nào cũng "công bằng" khi chọn ký tự có xác suất cao nhất, vì việc chọn ký tự có xác suất cao thứ hai vẫn có thể dẫn đến văn bản có ý nghĩa. Sẽ hợp lý hơn nếu chúng ta **lấy mẫu** ký tự từ phân bố xác suất do đầu ra của mạng cung cấp.

Việc lấy mẫu này có thể được thực hiện bằng hàm `np.multinomial`, hàm này triển khai cái gọi là **phân phối đa thức**. Một hàm thực hiện việc tạo văn bản **mềm** này được định nghĩa dưới đây:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Chúng tôi đã giới thiệu thêm một tham số gọi là **nhiệt độ**, được sử dụng để chỉ ra mức độ chúng ta nên tuân thủ xác suất cao nhất. Nếu nhiệt độ là 1.0, chúng ta thực hiện lấy mẫu đa thức công bằng, và khi nhiệt độ tiến tới vô cực - tất cả các xác suất trở nên bằng nhau, và chúng ta chọn ngẫu nhiên ký tự tiếp theo. Trong ví dụ dưới đây, chúng ta có thể quan sát rằng văn bản trở nên vô nghĩa khi chúng ta tăng nhiệt độ quá nhiều, và nó giống như văn bản được tạo cứng "lặp lại" khi nhiệt độ tiến gần đến 0.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
